In [1]:
!pip install  keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.3 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.15.0


In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [6]:
train_images = train_images/255.0
test_images = test_images/255.0

In [8]:
train_images[0].shape

(28, 28)

In [9]:
# resizing the images to the req size
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [10]:
train_images[0].shape

(28, 28, 1)

In [11]:
def build_model(hp):
  model = keras.Sequential(
      [
          keras.layers.Conv2D(
              filters = hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
              kernel_size = hp.Choice('conv_1_kernel', values=[3,5]),
              activation = 'relu',
              input_shape = (28,28,1)
          ),
          keras.layers.Conv2D(
              filters = hp.Int('conv_2_filter', min_value=32,max_value=64, step=16),
              kernel_size = hp.Choice('conv_1_kernel', values=[3,5]),
              activation = 'relu',
          ),
          keras.layers.Flatten(),
          keras.layers.Dense(
              units = hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
              activation = 'relu'
          ),
          keras.layers.Dense(10, activation='softmax')
      ]
  )

  model.compile(
      optimizer= keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2,1e-3])),
      loss = 'sparse_categorical_crossentropy',
      metrics=['accuracy']
      )

  return model


In [15]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-15-dce6bfdf082c>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [17]:
tuner_search = RandomSearch(build_model,
                            objective='val_accuracy',
                            max_trials=5,
                            directory='output',
                            project_name='mnist_fashion'
                            )

In [18]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 00m 26s]
val_accuracy: 0.9076666831970215

Best val_accuracy So Far: 0.9086666703224182
Total elapsed time: 00h 03m 05s


In [21]:
model = tuner_search.get_best_models(num_models=1)[0]

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 128)       1280      
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 48)        55344     
                                                                 
 flatten (Flatten)           (None, 27648)             0         
                                                                 
 dense (Dense)               (None, 96)                2654304   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 2711898 (10.35 MB)
Trainable params: 2711898 (10.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
model.fit(train_images, train_labels, epochs=15, validation_split=0.1, initial_epoch=3)

Epoch 4/15
1688/1688 [==============================] - 11s 6ms/step - loss: 0.1765 - accuracy: 0.9346 - val_loss: 0.2428 - val_accuracy: 0.9137
Epoch 5/15
1688/1688 [==============================] - 9s 6ms/step - loss: 0.1243 - accuracy: 0.9544 - val_loss: 0.3058 - val_accuracy: 0.9085
Epoch 6/15
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0857 - accuracy: 0.9685 - val_loss: 0.3101 - val_accuracy: 0.9152
Epoch 7/15
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0593 - accuracy: 0.9788 - val_loss: 0.3515 - val_accuracy: 0.9148
Epoch 8/15
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0400 - accuracy: 0.9854 - val_loss: 0.4039 - val_accuracy: 0.9138
Epoch 9/15
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0304 - accuracy: 0.9889 - val_loss: 0.4500 - val_accuracy: 0.9128
Epoch 10/15
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0246 - accuracy: 0.9915 - val_loss: 0.4566 - val_accura